In [ ]:
import pandas as pd
import numpy as np

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

import random

from scipy import stats
import seaborn as sns

from IPython.display import display
from collections import Counter

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.pyplot.style.use = "default"
font = {"family": "Verdana", 
       "weight": "normal"}


from matplotlib import rc
rc("font", **font)

matplotlib.rcParams["figure.figsize"]=(30.0,20.0)
matplotlib.rcParams["font.size"]=(40)

from scipy.spatial.distance import cdist as cd


In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

from scipy import misc
import pydotplus
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from PIL import Image

In [ ]:
calls_table = pd.read_csv(r"C:\Users\111\Documents\work\pyhton\calls.csv", sep = ";")
valid_rows = pd.read_csv(r"C:\Users\111\Documents\work\pyhton\valid_rows.csv", sep = ";", header=None)

df = calls_table[calls_table["ID"].isin(valid_rows[0])]
display(df.shape, df.head(10))


In [ ]:
target_vars_list = [x for x in df.columns if x.startswith("tar_")]
predictors_list = [y for y in df.columns if "predic_" in y]

In [ ]:
df.dropna(axis=0,how="any", inplace = True)

In [ ]:
pred = "predic_001"
plt.scatter(df.index,sorted(df[pred]))
axis = plt.gca()
axis.set_ylim([df[pred].min()-0.2,df[pred].max()+0.2])


In [ ]:
df[predictors_list].hist(bins=60,color = "m")
plt.show()

In [ ]:
df[target_vars_list].hist(bins=60,color = "m")
plt.show()

## Feature selection

In [ ]:
data_FS = df.copy(deep=True)
for i in data_FS[target_vars_list]:
    data_FS[i] = pd.cut(data_FS[i], bins=10,labels=False,right=True,retbins=False,precision=3, include_lowest=False)

In [ ]:
y = data_FS["predic_001"]
X = data_FS[predictors_list]

df = DecisionTreeClassifier(criterion="gini", min_samples_split = 15,max_features = "sqrt",presort=True)

dt.fit(X,y)
class_names = ["0","1","2","3","4","5","6","7","8","9"]
importances = dt.feature_impotances_
std = np.std(importances,axis=0)
val = pd.DataFrame(importances,index=predictors_list)
val.sort(columns=0,ascending=True)

a = val[val.values==0]
b = val[val.values!=0]

a.shape,b.shape

## PCA

In [ ]:
pca=PCA(n_components=17)

In [ ]:
pca.fit(df[predictors_list])

In [ ]:
fig = plt.figure(figsize=(20,15))
ax = fig.add_subplot(1,1,1)
plt.clf()
plt.plot(pca.explained_variance_,linewidth=2)
plt.axis("tight")
plt.xlabel("n_components")
plt.ylabel("explained_variance_")
plt.grad()
ax.set_xticks(np.arange(0,6,1))
plt.show()

## Диаграмма рессеяния

In [ ]:
cols = ["predic_001","predic_002","predic_003","tar_01","tar_02","tar_03"]
sns_plot = sns.pairplot(df[cols])
sns_plot.savefig("paiplot.png")

In [ ]:
from scipy.stats.stats import pearsonr

correl = []

for tar in target_vars_list:
    for pred in predictors_list:
        pears_cor,p_value = pearsonr(df[tar],df[pred])
        if p_value<0.05 and pears_cor>0.3:
            if pred not in correl:
                correl.append(pred)
            print("predictor: ",pred,"target:",tar,"Pearson_corr: ",pears_cor,"P-value: "p_value)